In [11]:
import pandas as pd
from sklearn2pmml import PMMLPipeline
from sklearn_pandas import DataFrameMapper
from sklearn2pmml import sklearn2pmml
from sklearn.model_selection import KFold, cross_val_score as CVS, train_test_split as TTS,GridSearchCV as cv
import numpy as np
from xgboost import XGBRegressor as XGBR
import pickle as pickle

In [87]:
# ['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
#        'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
#        'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
#        'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
#        'v_13', 'v_14']
test_df = pd.read_csv("used_car_testB_20200421.csv",sep=' ')
test_df['notRepairedDamage'] = list(map(lambda notRepairedDamage:'3.0' if notRepairedDamage=='-' else notRepairedDamage,
                                        test_df.notRepairedDamage.values
                                       )
                                   )
test_df['notRepairedDamage'].value_counts()

train_df = pd.read_csv("used_car_train_20200313.csv",sep=' ')
# 需要把‘notRepairedDamage’中的‘-’ 转换为数字
train_df['notRepairedDamage'] = list(map(lambda notRepairedDamage:'3.0' if notRepairedDamage =='-' else notRepairedDamage,
                                         train_df.notRepairedDamage.values
                                        )
                                    )
train_df['notRepairedDamage'].value_counts()


# 特征工程
# 首先需要把交易ID 和 name删除掉
train_df['trendData'] = train_df['creatDate'] - train_df['regDate']
train_df = train_df.drop(columns=['SaleID','name','regDate','creatDate'])
train_df = train_df.fillna(0)

test_df['trendData'] = test_df['creatDate'] - test_df['regDate']
submit_params_data = test_df.drop(columns=['SaleID','name','regDate','creatDate']) 
submit_params_data = submit_params_data.fillna(0)
submit_params_data

,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,...,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,trendData
0,67.0,0,1.0,0.0,0.0,101,15.0,0.0,5019,0,...,0.000241,0.105319,0.046233,0.094522,3.619512,-0.280607,-2.019761,0.978828,0.803322,159807
1,19.0,6,2.0,0.0,0.0,73,6.0,0.0,1505,0,...,0.000000,0.120323,0.046784,0.035385,2.997376,-1.406705,-1.020884,-1.349990,-0.200542,210099
2,5.0,5,4.0,0.0,0.0,120,5.0,3.0,1776,0,...,0.090836,0.000000,0.079655,0.073586,-3.951084,-0.433467,0.918964,1.634604,1.027173,69703
3,22.0,9,1.0,0.0,0.0,58,15.0,0.0,26,0,...,0.101777,0.098950,0.026830,0.096614,-2.846788,2.800267,-2.524610,1.076819,0.461610,139713
4,46.0,6,0.0,0.0,0.0,116,15.0,0.0,738,0,...,0.000000,0.066732,0.057771,0.068852,2.839010,-1.659801,-0.924142,0.199423,0.451014,130005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,4.0,4,0.0,0.0,1.0,150,15.0,3.0,5564,0,...,0.000292,0.141804,0.076393,0.039272,2.072901,-2.531869,1.716978,-1.063437,0.326587,119304
49996,65.0,1,0.0,0.0,0.0,179,4.0,0.0,5220,0,...,0.000991,0.155868,0.108425,0.067841,1.358504,-3.290295,4.269809,0.140524,0.556221,29914
49997,4.0,4,0.0,0.0,1.0,147,12.5,0.0,3795,0,...,0.000318,0.141872,0.071968,0.042966,2.165658,-2.417885,1.370612,-1.073133,0.270602,119105
49998,40.0,1,4.0,0.0,1.0,176,15.0,0.0,61,0,...,0.000023,0.067483,0.067526,0.009006,2.030114,-2.939244,0.569078,-1.718245,0.316379,139625


In [107]:
train_df.describe()

,model,brand,bodyType,fuelType,gearbox,power,kilometer,regionCode,seller,offerType,...,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,trendData
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.0,...,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.00000
mean,47.128707,8.052733,1.738527,0.354093,0.215973,119.316547,12.597160,2583.077267,0.000007,0.0,...,0.044923,0.124692,0.058144,0.061996,-0.001000,0.009035,0.004813,0.000313,-0.000688,126160.27964
std,49.536024,7.864956,1.760778,0.539747,0.411497,177.168419,3.919576,1885.363218,0.002582,0.0,...,0.051743,0.201410,0.029186,0.035692,3.772386,3.286071,2.517478,1.288988,1.038685,53650.12345
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,-9.168192,-5.558207,-9.639552,-4.153899,-6.546556,9097.00000
25%,10.000000,1.000000,0.000000,0.000000,0.000000,75.000000,12.500000,1018.000000,0.000000,0.0,...,0.000038,0.062474,0.035334,0.033930,-3.722303,-1.951543,-1.871846,-1.057789,-0.437034,89208.00000
50%,30.000000,6.000000,1.000000,0.000000,0.000000,110.000000,15.000000,2196.000000,0.000000,0.0,...,0.000812,0.095866,0.057014,0.058484,1.624076,-0.358053,-0.130753,-0.036245,0.141246,129401.00000
75%,66.000000,13.000000,3.000000,1.000000,0.000000,150.000000,15.000000,3843.000000,0.000000,0.0,...,0.102009,0.125243,0.079382,0.087491,2.844357,1.255022,1.776933,0.942813,0.680378,169399.00000
max,247.000000,39.000000,7.000000,6.000000,1.000000,19312.000000,15.000000,8120.000000,1.000000,0.0,...,0.151420,1.404936,0.160791,0.222787,12.357011,18.819042,13.847792,11.147669,8.658418,250402.00000


In [108]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 28 columns):
model                150000 non-null float64
brand                150000 non-null int64
bodyType             150000 non-null float64
fuelType             150000 non-null float64
gearbox              150000 non-null float64
power                150000 non-null int64
kilometer            150000 non-null float64
notRepairedDamage    150000 non-null object
regionCode           150000 non-null int64
seller               150000 non-null int64
offerType            150000 non-null int64
price                150000 non-null int64
v_0                  150000 non-null float64
v_1                  150000 non-null float64
v_2                  150000 non-null float64
v_3                  150000 non-null float64
v_4                  150000 non-null float64
v_5                  150000 non-null float64
v_6                  150000 non-null float64
v_7                  150000 non-null float64
v_

In [104]:
# 训练与测试数据准备
train_df_X = train_df[train_df.columns.difference(["price"])]
train_df_Y = train_df["price"]

test_size = 0.2
X_train,X_test,Y_train,Y_test = TTS(train_df_X.values,train_df_Y.values.astype(np.float64),
                                    test_size=test_size,
                                    random_state=30)
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
Y_train = Y_train.astype(np.float64)
Y_test = Y_test.astype(np.float64)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

best_max_depth = 15
best_learning_rate = 0.3
best_n_estimators = 15
# bese_min_child_weight = 0.8
reg = XGBR(max_depth = best_max_depth,
           learning_rate = best_learning_rate,
           n_estimators = best_n_estimators,
#            min_child_weight = bese_min_child_weight,
              early_stopping_rounds=5).fit(X_train,Y_train)
print("训练完毕")

print("训练集上的R^2得分：%s"%(reg.score(X_train,Y_train)))
print("训练集上的MAE得分：%s"%(np.mean(np.abs(reg.predict(X_train) -Y_train))))

print("测试集上的R^2得分：%s"%(reg.score(X_test,Y_test)))
print("测试集上的MAE得分：%s"%(np.mean(np.abs(reg.predict(X_test) -Y_test))))

(120000, 27) (120000,)
(30000, 27) (30000,)
训练完毕
训练集上的R^2得分：0.9975727325474745
训练集上的MAE得分：212.61654488126436
测试集上的R^2得分：0.9651157285697952
测试集上的MAE得分：625.0514751974741


In [105]:
# 存储最终预测结果
result = reg.predict(submit_params_data.values)
result_df = pd.DataFrame({"SaleID":test_df.SaleID.values,
                          "price":result
                         })
result_df.to_csv("submit_result.csv",sep=',',index=False)